# Tutorial showing the integration between the QDAC2 and the OPX to perform voltage sweeps using the QCoDeS framework

In this tutorial you will learn how to combine the QDAC2 with the OPX to perform n-dimensional voltage scans efficiently.

The trick here is to use a digital marker from the OPX to trigger the QDAC and make it step the output voltage of a given channel across a pre-loaded voltage list. Please note that the size of this list is limited to 65536 items per channel.

In this case, the latency induced by the communication between the control PC and the instruments and by transferring the data is suppressed since everything is controlled by the OPX pulse processor. The experimental runtime is then given by the QDAC bandwidth and the integration time. 

In the following examples, the OPX is program using its QCoDeS driver. You can learn more about its usage in the github (https://github.com/qua-platform/py-qua-tools/tree/main/qualang_tools/external_frameworks/qcodes) and usage examples (https://github.com/qua-platform/py-qua-tools/tree/main/examples/Qcodes_drivers/basic-driver).
The following can also be adapted to work without controlling the OPX through qcodes so that they match your current framework.

In [ ]:
import os
import time
import qcodes as qc
from qm.qua import *
from qcodes import initialise_or_create_database_at, load_or_create_experiment
from qcodes.utils.dataset.doNd import do1d, do0d
from qualang_tools.external_frameworks.qcodes.opx_driver import OPX
from qcodes_contrib_drivers.drivers.QDevil import QDAC2
from importlib import reload
from configuration import *

## QCoDeS set-up

In [ ]:
db_name = "QM_demo.db"  # Database name
sample_name = "demo"  # Sample name
exp_name = "OPX_QDAC2_integration"  # Experiment name

# Initialize qcodes database
db_file_path = os.path.join(os.getcwd(), db_name)
qc.config.core.db_location = db_file_path
initialise_or_create_database_at(db_file_path)
# Initialize the qcodes station to which instruments will be added
station = qc.Station()

In [ ]:
# Create the OPX instrument class
opx_instrument = OPX(config, name="OPX_demo", host="127.0.0.1", cluster_name="my_cluster")
# Add the OPX instrument to the qcodes station
station.add_component(opx_instrument)

In [ ]:
qdac2_ip = "127.0.0.1"
# Create the QDAC2 instrument class
qdac2 = QDAC2.QDac2("QDAC", visalib="@py", address=f"TCPIP::{qdac2_ip}::5025::SOCKET")
# Add the QDAC2 instrument to the qcodes station
station.add_component(qdac2)

In [ ]:
# Used to reload the configuration in order to force Jupyter to upload the existing variables
import configuration

reload(configuration)
from configuration import *

opx_instrument.config = config
opx_instrument.qmm = opx_instrument.qmm.open_qm(config)

## Example experiments:
1. 1D voltage sweep performed by the QDAC while the OPX is measuring using do1d
2. 1D voltage sweep performed by the QDAC which is triggered by the OPX which is also measuring using do0d
3. 2D voltage sweep where the fast axis is performed by the OPX while the slow axis is handled by the QDAC using do1d

### Experiment 1: 1D voltage sweep performed by the QDAC while the OPX is measuring using do1d

In this example, the QDAC is parametrized to output a constant voltage which is swept using the do1d qcodes method. Here the OPX is simply measuring at each level, but the QUA program can easily be modified to complexify the sequence.

This method is similar to how any external DC voltage source would be integrated with the OPX.

In [ ]:
n_avg = 100  # Number of averaging loops


### OPX section
def qdac_1d_sweep(simulate=False):
    with program() as prog:
        i = declare(int)
        n = declare(int)
        data = declare(fixed)
        data_st = declare_stream()

        with infinite_loop_():
            if not simulate:
                pause()
            with for_(n, 0, n < n_avg, n + 1):
                # Wait 1ms before measuring (depends of the QDAC filter option)
                wait(1_000_000 // 4, "readout_element")
                # Measure for 10µs with the OPX - Can be replaced by dual_demod, demod or else
                measure("readout", "readout_element", None, integration.full("const", data, "out1"))
                # Send the result to the stream processing
                save(data, data_st)

        with stream_processing():
            # Average all the data and save the values into "data".
            data_st.buffer(n_avg).map(FUNCTIONS.average()).save_all("data")
    return prog


# Pass the readout length (in ns) to the class to convert the demodulated/integrated data into Volts
opx_instrument.readout_pulse_length(readout_len)
# Add the custom sequence to the OPX
opx_instrument.qua_program = qdac_1d_sweep(simulate=False)

### QDAC2 section
qdac2.reset()  # Reset the qdac parameters to start from a blank instrument
ch1 = qdac2.channel(2)  # Define the QDAC2 channel
# Set the current range ("high" or "low") and filter ("dc": 10Hz ,  "med": 10khz,  "high": 300kHz)
ch1.output_mode(range="low", filter="med")
Vg1 = ch1.dc_constant_V  # Define the voltage parameter for this channel

### Run the experiment
experiment1 = load_or_create_experiment("qdac_1d_sweep_do1d", sample_name)

start_time = time.time()
do1d(
    Vg1,
    -1.5,
    1.5,
    11,
    0.001,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    # do_plot=True,
    exp=experiment1,
)
print(f"Elapsed time: {time.time() - start_time:.2f} s")
# Bring back the voltage to 0
Vg1(0)

### Experiment 2: 1D voltage sweep performed by the QDAC which is triggered by the OPX which is also measuring using do0d

In this example, the QDAC is parametrized to output the values defined in a pre-loaded voltage list. Stepping to the next value is done on the event of a digital marker sent by the OPX to one of the QDAC external trigger input. 
Here the OPX is simply triggering the QDAC and measuring at each level, but the QUA program can easily be modified to complexify the sequence.

To this purpose, an element called "qdac_trigger1" was created in the config and a digital pulse was defined. You can read more about digital waveform manipulations here: https://docs.quantum-machines.co/0.1/qm-qua-sdk/docs/Introduction/qua_overview/?h=digi#digital-waveform-manipulations

This method is very similar to the previous one but significantly reduces the runtime since it avoids the communication time and data fetching at each voltage step. The experimental runtime is then given by the QDAC bandwidth and the integration time. 

In [ ]:
n_avg = 100  # Number of averaging loops
# Voltage values in Volt
voltage_values1 = list(np.linspace(-1.5, 1.5, 101))


### OPX section
def qdac_1d_sweep_trig(simulate=False):
    with program() as prog:
        i = declare(int)
        n = declare(int)
        data = declare(fixed)
        data_st = declare_stream()

        with infinite_loop_():
            if not simulate:
                pause()
            with for_(n, 0, n < n_avg, n + 1):
                with for_(i, 0, i < len(voltage_values1), i + 1):
                    # Trigger the QDAC channel to output the next voltage level from the list
                    play("trig", "qdac_trigger1")
                    # Wait 1ms before measuring (depends of the QDAC filter option)
                    wait(1_000_000 // 4, "readout_element")
                    # Measure with the OPX
                    measure("readout", "readout_element", None, integration.full("const", data, "out1"))
                    # Send the result to the stream processing
                    save(data, data_st)

        with stream_processing():
            # Average all the data and save the values into "data".
            data_st.buffer(len(voltage_values1)).buffer(n_avg).map(FUNCTIONS.average()).save_all("data")
    return prog


# Pass the readout length (in ns) to the class to convert the demodulated/integrated data into Volts
opx_instrument.readout_pulse_length(readout_len)
# Set the setpoint for the loop performed on the OPX. Axis1 is the most inner non-averaging loop
opx_instrument.set_sweep_parameters("axis1", voltage_values1, "V", "Vg1")
# Add the custom sequence to the OPX
opx_instrument.qua_program = qdac_1d_sweep_trig(simulate=False)

### QDAC2 section
qdac2.reset()
ch1 = qdac2.channel(1)  # Define the QDAC2 channel
# Set the current range ("high" or "low") and filter ("dc": 10Hz ,  "med": 10khz,  "high": 300kHz)
ch1.output_mode(range="low", filter="med")
# Define the voltage list, stepping mode and dwell time
Vg1_list = ch1.dc_list(voltages=voltage_values1, stepped=True, dwell_s=5e-6)
# Set the trigger mode to external with input port "ext1"
Vg1_list.start_on_external(trigger=1)

### Run the experiment
experiment2 = load_or_create_experiment("qdac_1d_sweep_trig_do0d", sample_name)

start_time = time.time()
do0d(
    opx_instrument.run_exp,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    opx_instrument.halt,
    # do_plot=True,
    exp=experiment2,
)
print(f"Elapsed time: {time.time() - start_time:.2f} s")

# Bring the voltage back to 0
ch1.dc_constant_V(0)

### Experiment 3: 2D voltage sweep where the fast axis is scanned using the triggering method while the slow axis is handled by the QDAC using do1d

In this example, one channel of the QDAC is parametrized to step though a pre-loaded voltage list on the event of a digital marker provided by the OPX (fast axis). A second channel is set to output a constant voltage which will be scanned using the do1d qcodes method (slow axis).

The idea is for instance to acquire a charge stability map using a raster scan (for finding quantum dots).

This method is an extention of the previous ones showing how to sweep two voltage axes.

In [ ]:
n_avg = 100  # Number of averaging loops

# Voltage values in Volt
voltage_values1 = np.linspace(-0.4, 0.4, 51)
voltage_values2 = list(np.linspace(-1.5, 1.5, 51))

### QDAC2 section
qdac2.reset()  # Reset the qdac parameters
# Channel 1
ch1 = qdac2.channel(1)  # Define the QDAC2 channel
ch1.output_mode(range="low", filter="med")
# Define the voltage list, stepping mode and dwell time
Vg1_list = ch1.dc_list(voltages=voltage_values1, stepped=True, dwell_s=5e-6)
# Set the trigger mode to external with input port "ext1"
Vg1_list.start_on_external(trigger=1)
# Channel 2
ch2 = qdac2.channel(2)
ch2.output_mode(range="low", filter="med")
Vg2 = ch2.dc_constant_V  # Define the voltage parameter for this channel


### OPX section
def qdac_opx_combined(simulate=False):
    with program() as prog:
        i = declare(int)
        n = declare(int)
        data = declare(fixed)
        data_st = declare_stream()

        with infinite_loop_():
            if not simulate:
                pause()

            with for_(i, 0, i < len(voltage_values1), i + 1):
                # Trigger the QDAC channel to output the next voltage level from the list
                play("trig", "qdac_trigger1")
                # Wait 1ms before measuring (depends of the QDAC filter option)
                wait(1_000_000 // 4, "readout_element")
                # In this example the averaging is done on the most inner loop (single point averaging)
                with for_(n, 0, n < n_avg, n + 1):
                    # Measure with the OPX
                    measure("readout", "readout_element", None, integration.full("const", data, "out1"))
                    # Send the result to the stream processing
                    save(data, data_st)
            # Bring the voltage back to zero
            ramp_to_zero("gate")

        with stream_processing():
            # Average all the data and save the values into "data".
            data_st.buffer(n_avg).map(FUNCTIONS.average()).buffer(len(voltage_values1)).save_all("data")
    return prog


# Pass the readout length (in ns) to the class to convert the demodulated/integrated data into Volts
opx_instrument.readout_pulse_length(readout_len)
# Add the custom sequence to the OPX
opx_instrument.qua_program = qdac_opx_combined(simulate=False)
# Axis1 is the most inner non-averaging loop
opx_instrument.set_sweep_parameters("axis1", voltage_values1, "V", "Vg1")

### Run the experiment
experiment3 = load_or_create_experiment("Combined_2D_sweep_do1d", sample_name)

start_time = time.time()
do1d(
    Vg2,
    voltage_values2[0],
    voltage_values2[-1],
    len(voltage_values2),
    0.001,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    # do_plot=True,
    exp=experiment3,
)
print(f"Elapsed time: {time.time() - start_time:.2f} s")
# Bring the voltage back to 0
ch1.dc_constant_V(0)
Vg2(0)

### Experiment 4: 2D voltage sweep where both axes are scanned using the triggering method and the live plotting mode of the OPX is enabled

In this example, two channels of the QDAC are parametrized to step though two pre-loaded voltage lists on the event of two digital markers provided by the OPX (connected to ext1 and ext2). This method allows the fast acquisition of a 2D voltage map and the data can be fetched from the OPX in real time to enable live plotting (this assumes that the averaging is performed on the most outer loop and the program is only to be executed once (do0d only)).

In [ ]:
%matplotlib qt
n_avg = 10  # Number of averaging loops

# Voltage values in Volt
voltage_values1 = list(np.linspace(-0.4, 0.4, 51))
voltage_values2 = list(np.linspace(-1.5, 1.5, 21))

### QDAC2 section
qdac2.reset()  # Reset the qdac parameters
# Channel 1
ch1 = qdac2.channel(1)  # Define the QDAC2 channel
ch1.output_mode(range="low", filter="med")
# Define the voltage list, stepping mode and dwell time
Vg1_list = ch1.dc_list(voltages=voltage_values1, stepped=True, dwell_s=5e-6)
# Set the trigger mode to external with input port "ext1"
Vg1_list.start_on_external(trigger=1)
# Channel 2
ch2 = qdac2.channel(2)  # Define the QDAC2 channel
ch2.output_mode(range="low", filter="med")
# Define the voltage list, stepping mode and dwell time
Vg2_list = ch2.dc_list(voltages=voltage_values2, stepped=True, dwell_s=5e-6)
# Set the trigger mode to external with input port "ext2"
Vg2_list.start_on_external(trigger=2)


### OPX section
def qdac_opx_combined(simulate=False):
    with program() as prog:
        i = declare(int)
        j = declare(int)
        n = declare(int)
        data = declare(fixed)
        data_st = declare_stream()

        # No need for the infinite loop here since the program will only be executed once.
        # When using live plotting, the infinite_loop will prevent the program from ending because it blocks the python terminal
        # with infinite_loop_():
        if not simulate:
            pause()
        # In this example the averaging is done on the most outer loop to enable live plotting
        with for_(n, 0, n < n_avg, n + 1):
            with for_(i, 0, i < len(voltage_values2), i + 1):
                # Trigger the QDAC channel to output the next voltage level from the list
                play("trig", "qdac_trigger2")
                with for_(j, 0, j < len(voltage_values1), j + 1):
                    # Trigger the QDAC channel to output the next voltage level from the list
                    play("trig", "qdac_trigger1")
                    # Wait 1ms before measuring (depends of the QDAC filter option)
                    wait(1_000_000 // 4, "readout_element")
                    # Measure with the OPX
                    measure("readout", "readout_element", None, integration.full("const", data, "out1"))
                    # Send the result to the stream processing
                    save(data, data_st)

        with stream_processing():
            # When averaging on the most outer loop, if the program is meant to run once (do0d),
            # then the .average() method can be used for live plotting purposes
            data_st.buffer(len(voltage_values1)).buffer(len(voltage_values2)).average().save_all("data")
    return prog


# Pass the readout length (in ns) to the class to convert the demodulated/integrated data into Volts
opx_instrument.readout_pulse_length(readout_len)
# Add the custom sequence to the OPX
opx_instrument.qua_program = qdac_opx_combined(simulate=False)
# Axis1 is the most inner non-averaging loop
opx_instrument.set_sweep_parameters("axis1", voltage_values1, "V", "Vg1")
opx_instrument.set_sweep_parameters("axis2", voltage_values2, "V", "Vg2")

### Run the experiment
experiment4 = load_or_create_experiment("Combined_2D_sweep_do0d", sample_name)

start_time = time.time()
# Compile the QUA program and execute it
opx_instrument.run_exp()
# Exit the pause() statement and start the sequence
opx_instrument.resume()
# Uncomment to fetch the data in real-time and plot them
# opx_instrument.live_plotting(["data"])
# Update the data counter to save the last dataset to the qcodes database
opx_instrument.counter = n_avg
# Store the results in the qcodes database
do0d(
    opx_instrument.get_measurement_parameter(),
    do_plot=True,
    exp=experiment4,
)

print(f"Elapsed time: {time.time() - start_time:.2f} s")
# Bring the voltage back to 0
ch1.dc_constant_V(0)
ch2.dc_constant_V(0)

In [ ]:
import time
from typing import Dict, Optional

from qcodes import (
    Instrument,
    Parameter,
    MultiParameter,
)
from qcodes.utils.validators import Numbers, Arrays
from qm import SimulationConfig
from qm.qua import program
from qm.QuantumMachinesManager import QuantumMachinesManager
from qualang_tools.results import wait_until_job_is_paused
from qualang_tools.results import fetching_tool
from qualang_tools.plot import interrupt_on_close
import matplotlib.pyplot as plt
import numpy as np
import warnings

warnings.filterwarnings("ignore")


# noinspection PyAbstractClass
class OPX(Instrument):
    def __init__(
        self,
        config: Dict,
        name: str = "OPX",
        host: str = None,
        port: int = None,
        cluster_name: str = None,
        octave=None,
        close_other_machines: bool = True,
    ) -> None:
        """
        QCoDeS driver for the OPX.

        :param config: python dict containing the configuration expected by the OPX.
        :param name: The name of the instrument used internally by QCoDeS. Must be unique.
        :param host: IP address of the router to which the OPX is connected.
        :param cluster_name: Name of the cluster as defined in the OPX admin panel (from version QOP220)
        :param port: Port of the OPX or main OPX if working with a cluster.
        :param octave: Octave configuration if an Octave is to be used in this experiment.
        :param close_other_machines: Flag to control if opening a quantum machine will close the existing ones. Default is True. Set to False if multiple a QOP is to be used by multiple users or to run several experiments in parallel.
        """
        super().__init__(name)

        self.qm = None
        self.qmm = None
        self.config = None
        self.result_handles = None
        self.job = None
        self.counter = 0
        self.results = {"names": [], "types": [], "buffers": [], "units": []}
        self.prog_id = None
        self.simulated_wf = {}
        # Parameter for simulation duration
        self.add_parameter(
            "sim_time",
            unit="ns",
            label="sim_time",
            initial_value=100000,
            vals=Numbers(
                4,
            ),
            get_cmd=None,
            set_cmd=None,
        )
        self.measurement_variables = None
        self.add_parameter(
            "readout_pulse_length",
            unit="ns",
            vals=Numbers(16, 1e7),
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis1_stop",
            unit="",
            initial_value=0,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis1_start",
            unit="",
            initial_value=0,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis1_step",
            unit="",
            initial_value=0.1,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis1_npoints",
            unit="",
            initial_value=0,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis1_full_list",
            unit="",
            initial_value=0,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis1_axis",
            unit="",
            label="Axis 1",
            parameter_class=GeneratedSetPointsArbitrary,
            full_list=self.axis1_full_list,
            vals=Arrays(shape=(self.axis1_npoints.get_latest,)),
        )
        self.add_parameter(
            "axis2_stop",
            unit="",
            initial_value=0,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis2_start",
            unit="",
            initial_value=0,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis2_step",
            unit="",
            initial_value=0.1,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis2_npoints",
            unit="",
            initial_value=0,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis2_full_list",
            unit="",
            initial_value=0,
            get_cmd=None,
            set_cmd=None,
        )
        self.add_parameter(
            "axis2_axis",
            unit="",
            label="Axis 2",
            parameter_class=GeneratedSetPointsArbitrary,
            full_list=self.axis2_full_list,
            vals=Arrays(shape=(self.axis1_npoints.get_latest,)),
        )
        # Open QMM
        self.connect_to_qmm(host=host, port=port, cluster_name=cluster_name, octave=octave)
        # Set config
        self.set_config(config=config)
        # Open QM
        self.open_qm(close_other_machines)

    def connect_to_qmm(self, host: str = None, port: int = None, cluster_name: str = None, octave=None):
        """
        Enable the connection with the OPX by creating the QuantumMachineManager.
        Displays the connection message with idn when the connection is established.

        :param host: IP address of the router to which the OPX is connected.
        :param port: Port of the OPX or main OPX if working with a cluster.
        :param cluster_name: Name of the cluster as defined in the OPX admin panel (from version QOP220)
        :param octave: Octave configuration if an Octave is to be used in this experiment.
        """
        self.qmm = QuantumMachinesManager(host=host, port=port, cluster_name=cluster_name, octave=octave)
        self.connect_message()

    def connect_message(self, idn_param: str = "IDN", begin_time: Optional[float] = None) -> None:
        """
        Print a standard message on initial connection to an instrument.

        Args:
            idn_param: Name of parameter that returns ID dict.
                Default ``IDN``.
            begin_time: ``time.time()`` when init started.
                Default is ``self._t0``, set at start of ``Instrument.__init__``.
        """
        idn = {"vendor": "Quantum Machines"}
        idn.update(self.get(idn_param))
        if idn["server"][0] == "1":
            idn["model"] = "OPX"
        elif idn["server"][0] == "2":
            idn["model"] = "OPX+"
        else:
            idn["model"] = ""
        t = time.time() - (begin_time or self._t0)

        con_msg = (
            "Connected to: {vendor} {model} in {t:.2f}s. "
            "QOP Version = {server}, SDK Version = {client}.".format(t=t, **idn)
        )
        print(con_msg)
        self.log.info(f"Connected to instrument: {idn}")

    def get_idn(self) -> Dict[str, Optional[str]]:
        """
        Parse a standard VISA *IDN? response into an ID dict.

        :return: A dict containing the SDK version (client) and QOP version (server).
        """
        return self.qmm.version()

    def set_config(self, config):
        """
        Update the configuration used by the OPX.

        :param config: the new configuration.
        """
        self.config = config

    def open_qm(self, close_other_machines):
        """
        Open a quantum machine with a given configuration ready to execute a program.
        Beware that each call will close the existing quantum machines and interrupt the running jobs.
        """
        self.qm = self.qmm.open_qm(self.config, close_other_machines=close_other_machines)

    # Empty method that can be replaced by your pulse sequence in the main script
    # This can also be modified so that you can put the sequences here directly...
    def qua_program(self):
        """
        Custom QUA program

        :return: QUA program
        """
        with program() as prog:
            pass
        return prog

    # @abstractmethod
    def get_prog(self):
        """Get the QUA program from the user"""
        prog = self.qua_program
        return prog

    # @abstractmethod
    def get_res(self):
        """
        Fetch the results from the OPX, convert it into Volts and cast it into a dict.

        :return: dict containing the fetched results.
        """

        if self.result_handles is None:
            return None
        else:
            output = {}
            for i in range(len(self.results["types"])):
                # Get data and convert to Volt
                out = None
                # demodulated or integrated data
                self.result_handles.get(self.results["names"][i]).wait_for_values(self.counter)
                if self.results["types"][i] == "IQ":
                    out = (
                        -(self.result_handles.get(self.results["names"][i]).fetch(self.counter - 1)["value"])
                        * 4096
                        / self.readout_pulse_length()
                        * self.results["scale_factor"][i]
                    )
                # raw adc traces
                elif self.results["types"][i] == "adc":
                    out = (
                        -(self.result_handles.get(self.results["names"][i]).fetch(self.counter - 1)["value"])
                        / 4096
                        * self.results["scale_factor"][i]
                    )
                # Reshape data
                if len(self.results["buffers"][i]) == 2:
                    output[self.results["names"][i]] = out.reshape(
                        self.results["buffers"][i][0], self.results["buffers"][i][1]
                    )
                elif len(self.results["buffers"][i]) == 1:
                    output[self.results["names"][i]] = out.reshape(self.results["buffers"][i][0])
                else:
                    output[self.results["names"][i]] = out

            # Add amplitude and phase if I and Q are in the SP
            if "I" in output.keys() and "Q" in output.keys():
                output["R"] = np.sqrt(output["I"] ** 2 + output["Q"] ** 2)
                output["Phi"] = np.unwrap(np.angle(output["I"] + 1j * output["Q"])) * 180 / np.pi
            return output

    def _extend_result(self, gene, count, averaging_buffer):
        """
        Recursive function to get relevant information from the stream processing to construct the result Parameter.

        :param gene: stream processing generator (item of prog.result_analysis._result_analysis.model).
        :param count: counter to keep track of the buffers for a given stream.
        :param averaging_buffer: flag identifying if a buffer is used for averaging.
        """

        if len(gene.values) > 0:
            if gene.values[0].string_value == "saveAll":
                self.results["names"].append(gene.values[1].string_value)
                self.results["types"].append("IQ")
                self.results["units"].append("V")
                self.results["buffers"].append([])
                self.results["scale_factor"].append(1)
                # Check if next buffer is for averaging
                if len(gene.values[2].list_value.values[1].list_value.values) > 0:
                    # print(gene.values[2].list_value.values[1].list_value.values[0].string_value)
                    if gene.values[2].list_value.values[1].list_value.values[0].string_value == "average":
                        averaging_buffer = True
            elif gene.values[0].string_value == "buffer":
                if not averaging_buffer:
                    self.results["buffers"][count].append(int(gene.values[1].string_value))
                    # Check if next buffer is for averaging
                    if len(gene.values[2].list_value.values[1].list_value.values) > 0:
                        if gene.values[2].list_value.values[1].list_value.values[0].string_value == "average":
                            averaging_buffer = True
                else:
                    averaging_buffer = False
            elif gene.values[0].string_value == "@macro_adc_trace":
                self.results["buffers"][count].append(int(self.readout_pulse_length()))
                self.results["types"][count] = "adc"
                self.results["scale_factor"].append(1)
            else:
                pass
            if len(gene.values) > 2:
                self._extend_result(gene.values[2].list_value, count, averaging_buffer)
            elif gene.values[0].string_value == "average":
                self._extend_result(gene.values[1].list_value, count, averaging_buffer)

    def _get_stream_processing(self, prog):
        """
        Get relevant information from the stream processing to construct the result Parameter.

        :param prog: QUA program.
        """
        count = 0
        for i in prog.result_analysis._result_analysis.model:
            self._extend_result(i, count, False)
            count += 1

    def set_sweep_parameters(self, scanned_axis, setpoints, unit=None, label=None):
        """
        Set the setpoint parameters.

        :param scanned_axis: Can be axis1 for the most inner loop and axis2 for the outer one. 3 dimensional scans or higher are not implemented.
        :param setpoints: Values of the sweep parameter as a python list.
        :param unit: Unit of the setpoint ("V" for instance).
        :param label: Label of the setpoint ("Bias voltage" for instance).
        """
        if scanned_axis == "axis1":
            self.axis1_full_list(setpoints)
            self.axis1_start(setpoints[0])
            self.axis1_stop(setpoints[-1])
            self.axis1_step(setpoints[1] - setpoints[0])
            self.axis1_npoints(len(setpoints))
            self.axis1_axis.unit = unit
            self.axis1_axis.label = label
        elif scanned_axis == "axis2":
            self.axis2_full_list(setpoints)
            self.axis2_start(setpoints[0])
            self.axis2_stop(setpoints[-1])
            self.axis2_step(setpoints[1] - setpoints[0])
            self.axis2_npoints(len(setpoints))
            self.axis2_axis.unit = unit
            self.axis2_axis.label = label

    def get_measurement_parameter(self, scale_factor=((),)):
        """
        Find the correct Parameter shape based on the stream-processing and return the measurement Parameter.

        :param scale_factor: list of tuples containing the parameter to rescale, the scale factor with respect to Volts and the new unit as in scale_factor=[(I, 0.152, "pA"), (Q, 0.152, "pA")].
        :return: Qcodes measurement parameters.
        """

        # Reset the results in case the stream processing was changed between two iterations
        self.results = {
            "names": [],
            "types": [],
            "buffers": [],
            "units": [],
            "scale_factor": [],
        }
        # Add amplitude and phase if I and Q are in the SP
        if len(self.results["names"]) == 0:
            self._get_stream_processing(self.get_prog())

            if "I" in self.results["names"] and "Q" in self.results["names"]:
                self.results["names"].append("R")
                self.results["names"].append("Phi")
                self.results["units"].append("V")
                self.results["units"].append("rad")
            if "adc" in self.results["types"]:
                self.axis1_start(0)
                self.axis1_stop(int(self.readout_pulse_length()))
                self.axis1_step(1)
                self.axis1_npoints(int(self.readout_pulse_length()))
                self.axis1_full_list(np.arange(self.axis1_start(), self.axis1_stop(), self.axis1_step()))
                self.axis1_axis.unit = "ns"
                self.axis1_axis.label = "Readout time"
        # Rescale the results if a scale factor is provided
        if len(scale_factor) > 0:
            if len(scale_factor[0]) > 0:
                if len(scale_factor[0]) == 3:
                    for param in scale_factor:
                        if param[0] in self.results["names"]:
                            self.results["units"][self.results["names"].index(param[0])] = param[2]
                            self.results["scale_factor"][self.results["names"].index(param[0])] = param[1]
                else:
                    raise ValueError(
                        "scale_factor must be a list of tuples with 3 elements (the result name, the scale factor and the new unit), as in [('I', 0.152, 'pA'), ]."
                    )
        if len(self.results["buffers"]) > 0 and len(self.results["buffers"][0]) > 0:
            if len(self.results["buffers"][0]) == 2:
                return ResultParameters(
                    self,
                    self.results["names"],
                    "OPX_results",
                    names=self.results["names"],
                    units=self.results["units"],
                    shapes=((self.results["buffers"][0][0], self.results["buffers"][0][1]),)
                    * len(self.results["names"]),
                    setpoints=((self.axis2_axis(), self.axis1_axis()),) * len(self.results["names"]),
                    setpoint_units=((self.axis2_axis.unit, self.axis1_axis.unit),) * len(self.results["names"]),
                    setpoint_labels=((self.axis2_axis.label, self.axis1_axis.label),) * len(self.results["names"]),
                    setpoint_names=(
                        (
                            self.axis2_axis.label.replace(" ", "").lower(),
                            self.axis1_axis.label.replace(" ", "").lower(),
                        ),
                    )
                    * len(self.results["names"]),
                )
            elif len(self.results["buffers"][0]) == 1:
                return ResultParameters(
                    self,
                    self.results["names"],
                    "OPX_results",
                    names=self.results["names"],
                    units=self.results["units"],
                    shapes=((self.results["buffers"][0][0],),) * len(self.results["names"]),
                    setpoints=((self.axis1_axis(),),) * len(self.results["names"]),
                    setpoint_units=((self.axis1_axis.unit,),) * len(self.results["names"]),
                    setpoint_labels=((self.axis1_axis.label,),) * len(self.results["names"]),
                    setpoint_names=((self.axis1_axis.label.replace(" ", "").lower(),),) * len(self.results["names"]),
                )
        else:
            return ResultParameters(
                self,
                self.results["names"],
                "OPX_results",
                names=self.results["names"],
                units=self.results["units"],
                shapes=((),) * len(self.results["names"]),
                setpoints=((),) * len(self.results["names"]),
                setpoint_units=((),) * len(self.results["names"]),
                setpoint_labels=((),) * len(self.results["names"]),
            )

    def live_plotting(self, results_to_plot: list = ()):
        # Get the plotting grid
        if len(results_to_plot) == 0:
            raise ValueError("At least 1 result to plot must be provided")
        elif len(results_to_plot) == 1:
            grid = 1
        elif len(results_to_plot) == 2:
            grid = 121
        elif len(results_to_plot) == 3 or len(results_to_plot) == 4:
            grid = 221
        else:
            raise ValueError("Live plotting is limited to 4 parameters.")
        # Get results from QUA program
        results = fetching_tool(self.job, results_to_plot, "live")
        # Live plotting
        fig = plt.figure()
        interrupt_on_close(fig, self.job)  # Interrupts the job when closing the figure
        while results.is_processing():
            # Fetch results
            data_list = results.fetch_all()
            # Subplot counter
            i = 0
            for data in data_list:
                # Convert the results into Volts
                data = -data[-1] * 4096 / int(self.readout_pulse_length())
                # Plot results
                if len(data.shape) == 1:
                    if len(results_to_plot) > 1:
                        plt.subplot(grid + i)
                    plt.cla()
                    plt.plot(self.axis1_axis(), data)
                    plt.xlabel(self.axis1_axis.label + f" [{self.axis1_axis.unit}]")
                    plt.ylabel(results_to_plot[i] + " [V]")
                elif len(data.shape) == 2:
                    if len(results_to_plot) > 1:
                        plt.subplot(grid + i)
                    plt.cla()
                    plt.pcolor(self.axis1_axis(), self.axis2_axis(), data)
                    plt.xlabel(self.axis1_axis.label + f" [{self.axis1_axis.unit}]")
                    plt.ylabel(self.axis2_axis.label + f" [{self.axis2_axis.unit}]")
                    plt.title(results_to_plot[i] + " [V]")
                i += 1
            plt.pause(1)
            plt.tight_layout()

    def run_exp(self):
        """
        Execute a given QUA program, initialize the counter to 0 and creates a result handle to fetch the results.
        """
        prog = self.get_prog()
        self.job = self.qm.execute(prog)
        self.counter = 0
        self.result_handles = self.job.result_handles

    def resume(self, timeout: int = 30):
        """
        Resume the job and increment the counter to keep track of the fetched results.

        :param timeout: duration in seconds after which the console will be freed even if the pause statement has not been reached to prevent from being stuck here forever.
        """
        wait_until_job_is_paused(self.job, timeout)
        if not self.job.is_paused():
            raise RuntimeError(f"The program has not reached the pause statement before {timeout} s.")
        else:
            self.job.resume()
            self.counter += 1

    def compile_prog(self, prog):
        """
        Compile a given QUA program and stores it under the prog_id attribute.

        :param prog: A QUA program to be compiled.
        """
        self.prog_id = self.qm.compile(prog)

    def execute_compiled_prog(self):
        """
        Add a compiled program to the current queue and create a result handle to fetch the results.
        """
        if self.prog_id is not None:
            pending_job = self.qm.queue.add_compiled(self.prog_id)
            self.job = pending_job.wait_for_execution()
            self.result_handles = self.job.result_handles

    def simulate(self):
        """
        Simulate a given QUA program and store the simulated waveform into the simulated_wf attribute.
        """
        prog = self.get_prog()
        self.job = self.qmm.simulate(self.config, prog, SimulationConfig(self.sim_time() // 4))
        simulated_samples = self.job.get_simulated_samples()
        for con in [f"con{i}" for i in range(1, 10)]:
            if hasattr(simulated_samples, con):
                self.simulated_wf[con] = {}
                self.simulated_wf[con]["analog"] = self.job.get_simulated_samples().__dict__[con].analog
                self.simulated_wf[con]["digital"] = self.job.get_simulated_samples().__dict__[con].digital
        self.result_handles = self.job.result_handles

    def plot_simulated_wf(self):
        """
        Plot the simulated waveforms in a new figure using matplotlib.
        """
        plt.figure()
        for con in self.simulated_wf.keys():
            for t in self.simulated_wf[con].keys():
                for port in self.simulated_wf[con][t].keys():
                    if not np.all(self.simulated_wf[con][t][port] == 0):
                        if len(self.simulated_wf.keys()) == 1:
                            plt.plot(self.simulated_wf[con][t][port], label=f"{t} {port}")
                        else:
                            plt.plot(
                                self.simulated_wf[con][t][port],
                                label=f"{con} {t} {port}",
                            )
        plt.xlabel("Time [ns]")
        plt.ylabel("Voltage level [V]")
        plt.title("Simulated waveforms")
        plt.grid("on")
        plt.legend()

    def close(self) -> None:
        """
        Close the quantum machine and tear down the OPX instrument.
        """
        if self.qm is not None:
            self.qm.close()
        super().close()

    def halt(self) -> None:
        """
        Interrupt the current job and halt the running program.
        """
        if self.job is not None:
            self.job.halt()


# noinspection PyAbstractClass
class ResultParameters(MultiParameter):
    """
    Subclass of MultiParameter suited for the results acquired with the OPX.
    """

    def __init__(
        self,
        instr,
        params,
        name,
        names,
        units,
        shapes=None,
        setpoints=None,
        *args,
        **kwargs,
    ):
        super().__init__(
            name=name,
            names=names,
            units=units,
            shapes=shapes,
            setpoints=setpoints,
            *args,
            **kwargs,
        )

        self._instr = instr
        self._params = params

    def get_raw(self):
        vals = []
        result = self._instr.get_res()
        for param in self._params:
            if param in self._instr.results["names"]:
                vals.append(result[param])
        return tuple(vals)


# noinspection PyAbstractClass
class GeneratedSetPoints(Parameter):
    """
    A parameter that generates a setpoint array from start, stop and num points
    parameters.
    """

    def __init__(self, startparam, stopparam, numpointsparam, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._startparam = startparam
        self._stopparam = stopparam
        self._numpointsparam = numpointsparam

    def get_raw(self):
        return np.linspace(self._startparam(), self._stopparam(), self._numpointsparam())


# noinspection PyAbstractClass
class GeneratedSetPointsSpan(Parameter):
    """
    A parameter that generates a setpoint array from center, span and num points
    parameters.
    """

    def __init__(self, spanparam, centerparam, numpointsparam, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._spanparam = spanparam
        self._centerparam = centerparam
        self._numpointsparam = numpointsparam

    def get_raw(self):
        return np.linspace(
            self._centerparam() - self._spanparam() / 2,
            self._centerparam() + self._spanparam() / 2,
            self._numpointsparam(),
        )


# noinspection PyAbstractClass
class GeneratedSetPointsArbitrary(Parameter):
    """
    A parameter that generates a setpoint array from an arbitrary list of points.
    """

    def __init__(self, full_list, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.full_list = full_list

    def get_raw(self):
        return self.full_list()